## Predicting FIFA Man of the Match

Data source: https://www.kaggle.com/datasets/mathan/fifa-2018-match-statistics

In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier
import tensorflow as tf

In [6]:
data = pd.read_csv('archive/FIFA 2018 Statistics.csv')
data

,Date,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,14-06-2018,Russia,Saudi Arabia,5,40,13,7,3,3,6,3,11,0,78,306,118,22,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,14-06-2018,Saudi Arabia,Russia,0,60,6,0,3,3,2,1,25,2,86,511,105,10,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,15-06-2018,Egypt,Uruguay,0,43,8,3,3,2,0,1,7,3,78,395,112,12,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,15-06-2018,Uruguay,Egypt,1,57,14,4,6,4,5,1,13,3,86,589,111,6,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,15-06-2018,Morocco,Iran,0,64,13,3,6,4,5,0,14,2,86,433,101,22,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,11-07-2018,England,Croatia,1,46,11,1,6,4,4,3,24,5,79,479,148,14,1,0,0,No,5.0,Semi- Finals,No,0,NaN,NaN
124,14-07-2018,Belgium,England,2,43,12,4,3,5,4,1,5,5,88,510,108,11,1,0,0,Yes,4.0,3rd Place,No,0,NaN,NaN
125,14-07-2018,England,Belgium,0,57,15,5,7,3,5,0,12,2,92,698,110,5,2,0,0,No,NaN,3rd Place,No,0,NaN,NaN
126,15-07-2018,France,Croatia,4,39,8,6,1,1,2,1,14,1,75,271,99,14,2,0,0,Yes,18.0,Final,No,0,1.0,18.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 27 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    128 non-null    object 
 1   Team                    128 non-null    object 
 2   Opponent                128 non-null    object 
 3   Goal Scored             128 non-null    int64  
 4   Ball Possession %       128 non-null    int64  
 5   Attempts                128 non-null    int64  
 6   On-Target               128 non-null    int64  
 7   Off-Target              128 non-null    int64  
 8   Blocked                 128 non-null    int64  
 9   Corners                 128 non-null    int64  
 10  Offsides                128 non-null    int64  
 11  Free Kicks              128 non-null    int64  
 12  Saves                   128 non-null    int64  
 13  Pass Accuracy %         128 non-null    int64  
 14  Passes                  128 non-null    in

### Preprocessing

In [23]:
df = data.copy()

In [24]:
df = df.drop('Date', axis=1)

In [25]:
df

,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Own goals,Own goal Time
0,Russia,Saudi Arabia,5,40,13,7,3,3,6,3,11,0,78,306,118,22,0,0,0,Yes,12.0,Group Stage,No,0,NaN,NaN
1,Saudi Arabia,Russia,0,60,6,0,3,3,2,1,25,2,86,511,105,10,0,0,0,No,NaN,Group Stage,No,0,NaN,NaN
2,Egypt,Uruguay,0,43,8,3,3,2,0,1,7,3,78,395,112,12,2,0,0,No,NaN,Group Stage,No,0,NaN,NaN
3,Uruguay,Egypt,1,57,14,4,6,4,5,1,13,3,86,589,111,6,0,0,0,Yes,89.0,Group Stage,No,0,NaN,NaN
4,Morocco,Iran,0,64,13,3,6,4,5,0,14,2,86,433,101,22,1,0,0,No,NaN,Group Stage,No,0,1.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,England,Croatia,1,46,11,1,6,4,4,3,24,5,79,479,148,14,1,0,0,No,5.0,Semi- Finals,No,0,NaN,NaN
124,Belgium,England,2,43,12,4,3,5,4,1,5,5,88,510,108,11,1,0,0,Yes,4.0,3rd Place,No,0,NaN,NaN
125,England,Belgium,0,57,15,5,7,3,5,0,12,2,92,698,110,5,2,0,0,No,NaN,3rd Place,No,0,NaN,NaN
126,France,Croatia,4,39,8,6,1,1,2,1,14,1,75,271,99,14,2,0,0,Yes,18.0,Final,No,0,1.0,18.0


#### Dealing with missing values

In [26]:
df.isnull().sum()

Team                        0
Opponent                    0
Goal Scored                 0
Ball Possession %           0
Attempts                    0
On-Target                   0
Off-Target                  0
Blocked                     0
Corners                     0
Offsides                    0
Free Kicks                  0
Saves                       0
Pass Accuracy %             0
Passes                      0
Distance Covered (Kms)      0
Fouls Committed             0
Yellow Card                 0
Yellow & Red                0
Red                         0
Man of the Match            0
1st Goal                   34
Round                       0
PSO                         0
Goals in PSO                0
Own goals                 116
Own goal Time             116
dtype: int64

In [27]:
df = df.drop(['Own goals', 'Own goal Time'], axis=1)

In [28]:
df.isna().sum()

Team                       0
Opponent                   0
Goal Scored                0
Ball Possession %          0
Attempts                   0
On-Target                  0
Off-Target                 0
Blocked                    0
Corners                    0
Offsides                   0
Free Kicks                 0
Saves                      0
Pass Accuracy %            0
Passes                     0
Distance Covered (Kms)     0
Fouls Committed            0
Yellow Card                0
Yellow & Red               0
Red                        0
Man of the Match           0
1st Goal                  34
Round                      0
PSO                        0
Goals in PSO               0
dtype: int64

In [29]:
df['1st Goal'] = df['1st Goal'].fillna(df['1st Goal'].mean())

In [30]:
df.isna().sum().sum()

np.int64(0)

#### Encoding

In [31]:
df.dtypes

Team                       object
Opponent                   object
Goal Scored                 int64
Ball Possession %           int64
Attempts                    int64
On-Target                   int64
Off-Target                  int64
Blocked                     int64
Corners                     int64
Offsides                    int64
Free Kicks                  int64
Saves                       int64
Pass Accuracy %             int64
Passes                      int64
Distance Covered (Kms)      int64
Fouls Committed             int64
Yellow Card                 int64
Yellow & Red                int64
Red                         int64
Man of the Match           object
1st Goal                  float64
Round                      object
PSO                        object
Goals in PSO                int64
dtype: object

In [32]:
print(f"Team: {df['Team'].unique()}\n")
print(f"Opponent: {df['Opponent'].unique()}\n")
print(f"Man of the Match: {df['Man of the Match'].unique()}\n")
print(f"Round: {df['Round'].unique()}\n")
print(f"PSO: {df['PSO'].unique()}\n")

Team: ['Russia' 'Saudi Arabia' 'Egypt' 'Uruguay' 'Morocco' 'Iran' 'Portugal'
 'Spain' 'France' 'Australia' 'Argentina' 'Iceland' 'Peru' 'Denmark'
 'Croatia' 'Nigeria' 'Costa Rica' 'Serbia' 'Germany' 'Mexico' 'Brazil'
 'Switzerland' 'Sweden' 'Korea Republic' 'Belgium' 'Panama' 'Tunisia'
 'England' 'Colombia' 'Japan' 'Poland' 'Senegal']

Opponent: ['Saudi Arabia' 'Russia' 'Uruguay' 'Egypt' 'Iran' 'Morocco' 'Spain'
 'Portugal' 'Australia' 'France' 'Iceland' 'Argentina' 'Denmark' 'Peru'
 'Nigeria' 'Croatia' 'Serbia' 'Costa Rica' 'Mexico' 'Germany'
 'Switzerland' 'Brazil' 'Korea Republic' 'Sweden' 'Panama' 'Belgium'
 'England' 'Tunisia' 'Japan' 'Colombia' 'Senegal' 'Poland']

Man of the Match: ['Yes' 'No']

Round: ['Group Stage' 'Round of 16' 'Quarter Finals' 'Semi- Finals' '3rd Place'
 'Final']

PSO: ['No' 'Yes']



In [33]:
label_encoder = LabelEncoder()
df['Man of the Match'] = label_encoder.fit_transform(df['Man of the Match'])
man_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}
man_mappings

{0: 'No', 1: 'Yes'}

In [34]:
df['PSO'] = label_encoder.fit_transform(df['PSO'])
pso_mappings = {index: label for index, label in enumerate(label_encoder.classes_)}
pso_mappings

{0: 'No', 1: 'Yes'}

In [35]:
df

,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO
0,Russia,Saudi Arabia,5,40,13,7,3,3,6,3,11,0,78,306,118,22,0,0,0,1,12.000000,Group Stage,0,0
1,Saudi Arabia,Russia,0,60,6,0,3,3,2,1,25,2,86,511,105,10,0,0,0,0,39.457447,Group Stage,0,0
2,Egypt,Uruguay,0,43,8,3,3,2,0,1,7,3,78,395,112,12,2,0,0,0,39.457447,Group Stage,0,0
3,Uruguay,Egypt,1,57,14,4,6,4,5,1,13,3,86,589,111,6,0,0,0,1,89.000000,Group Stage,0,0
4,Morocco,Iran,0,64,13,3,6,4,5,0,14,2,86,433,101,22,1,0,0,0,39.457447,Group Stage,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,England,Croatia,1,46,11,1,6,4,4,3,24,5,79,479,148,14,1,0,0,0,5.000000,Semi- Finals,0,0
124,Belgium,England,2,43,12,4,3,5,4,1,5,5,88,510,108,11,1,0,0,1,4.000000,3rd Place,0,0
125,England,Belgium,0,57,15,5,7,3,5,0,12,2,92,698,110,5,2,0,0,0,39.457447,3rd Place,0,0
126,France,Croatia,4,39,8,6,1,1,2,1,14,1,75,271,99,14,2,0,0,1,18.000000,Final,0,0


In [36]:
round_values = list(df['Round'].unique())
round_values

['Group Stage',
 'Round of 16',
 'Quarter Finals',
 'Semi- Finals',
 '3rd Place',
 'Final']

In [37]:
round_mappings = {label: index for index, label in enumerate(round_values)}
round_mappings

{'Group Stage': 0,
 'Round of 16': 1,
 'Quarter Finals': 2,
 'Semi- Finals': 3,
 '3rd Place': 4,
 'Final': 5}

In [39]:
df['Round'] = df['Round'].apply(lambda x: round_mappings[x])

In [40]:
df

,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO
0,Russia,Saudi Arabia,5,40,13,7,3,3,6,3,11,0,78,306,118,22,0,0,0,1,12.000000,0,0,0
1,Saudi Arabia,Russia,0,60,6,0,3,3,2,1,25,2,86,511,105,10,0,0,0,0,39.457447,0,0,0
2,Egypt,Uruguay,0,43,8,3,3,2,0,1,7,3,78,395,112,12,2,0,0,0,39.457447,0,0,0
3,Uruguay,Egypt,1,57,14,4,6,4,5,1,13,3,86,589,111,6,0,0,0,1,89.000000,0,0,0
4,Morocco,Iran,0,64,13,3,6,4,5,0,14,2,86,433,101,22,1,0,0,0,39.457447,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,England,Croatia,1,46,11,1,6,4,4,3,24,5,79,479,148,14,1,0,0,0,5.000000,3,0,0
124,Belgium,England,2,43,12,4,3,5,4,1,5,5,88,510,108,11,1,0,0,1,4.000000,4,0,0
125,England,Belgium,0,57,15,5,7,3,5,0,12,2,92,698,110,5,2,0,0,0,39.457447,4,0,0
126,France,Croatia,4,39,8,6,1,1,2,1,14,1,75,271,99,14,2,0,0,1,18.000000,5,0,0


In [45]:
df['Team'].unique(), df['Opponent'].unique()

(array(['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay', 'Morocco', 'Iran',
        'Portugal', 'Spain', 'France', 'Australia', 'Argentina', 'Iceland',
        'Peru', 'Denmark', 'Croatia', 'Nigeria', 'Costa Rica', 'Serbia',
        'Germany', 'Mexico', 'Brazil', 'Switzerland', 'Sweden',
        'Korea Republic', 'Belgium', 'Panama', 'Tunisia', 'England',
        'Colombia', 'Japan', 'Poland', 'Senegal'], dtype=object),
 array(['Saudi Arabia', 'Russia', 'Uruguay', 'Egypt', 'Iran', 'Morocco',
        'Spain', 'Portugal', 'Australia', 'France', 'Iceland', 'Argentina',
        'Denmark', 'Peru', 'Nigeria', 'Croatia', 'Serbia', 'Costa Rica',
        'Mexico', 'Germany', 'Switzerland', 'Brazil', 'Korea Republic',
        'Sweden', 'Panama', 'Belgium', 'England', 'Tunisia', 'Japan',
        'Colombia', 'Senegal', 'Poland'], dtype=object))

In [44]:
pd.get_dummies(df['Team'], dtype=int)

,Argentina,Australia,Belgium,Brazil,Colombia,Costa Rica,Croatia,Denmark,Egypt,England,France,Germany,Iceland,Iran,Japan,Korea Republic,Mexico,Morocco,Nigeria,Panama,Peru,Poland,Portugal,Russia,Saudi Arabia,Senegal,Serbia,Spain,Sweden,Switzerland,Tunisia,Uruguay
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
124,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
df['Opponent'] = df['Opponent'].apply(lambda x: 'opp_' + x)
pd.get_dummies(df['Opponent'], dtype=int)

,opp_Argentina,opp_Australia,opp_Belgium,opp_Brazil,opp_Colombia,opp_Costa Rica,opp_Croatia,opp_Denmark,opp_Egypt,opp_England,opp_France,opp_Germany,opp_Iceland,opp_Iran,opp_Japan,opp_Korea Republic,opp_Mexico,opp_Morocco,opp_Nigeria,opp_Panama,opp_Peru,opp_Poland,opp_Portugal,opp_Russia,opp_Saudi Arabia,opp_Senegal,opp_Serbia,opp_Spain,opp_Sweden,opp_Switzerland,opp_Tunisia,opp_Uruguay
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
124,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
125,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
126,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [49]:
df

,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO
0,Russia,opp_Saudi Arabia,5,40,13,7,3,3,6,3,11,0,78,306,118,22,0,0,0,1,12.000000,0,0,0
1,Saudi Arabia,opp_Russia,0,60,6,0,3,3,2,1,25,2,86,511,105,10,0,0,0,0,39.457447,0,0,0
2,Egypt,opp_Uruguay,0,43,8,3,3,2,0,1,7,3,78,395,112,12,2,0,0,0,39.457447,0,0,0
3,Uruguay,opp_Egypt,1,57,14,4,6,4,5,1,13,3,86,589,111,6,0,0,0,1,89.000000,0,0,0
4,Morocco,opp_Iran,0,64,13,3,6,4,5,0,14,2,86,433,101,22,1,0,0,0,39.457447,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,England,opp_Croatia,1,46,11,1,6,4,4,3,24,5,79,479,148,14,1,0,0,0,5.000000,3,0,0
124,Belgium,opp_England,2,43,12,4,3,5,4,1,5,5,88,510,108,11,1,0,0,1,4.000000,4,0,0
125,England,opp_Belgium,0,57,15,5,7,3,5,0,12,2,92,698,110,5,2,0,0,0,39.457447,4,0,0
126,France,opp_Croatia,4,39,8,6,1,1,2,1,14,1,75,271,99,14,2,0,0,1,18.000000,5,0,0


In [50]:
df_concat = pd.concat([df, pd.get_dummies(df['Team']), pd.get_dummies(df['Opponent'])], axis=1)

In [51]:
df_concat

,Team,Opponent,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Argentina,Australia,Belgium,Brazil,Colombia,Costa Rica,Croatia,Denmark,Egypt,England,France,Germany,Iceland,Iran,Japan,Korea Republic,Mexico,Morocco,Nigeria,Panama,Peru,Poland,Portugal,Russia,Saudi Arabia,Senegal,Serbia,Spain,Sweden,Switzerland,Tunisia,Uruguay,opp_Argentina,opp_Australia,opp_Belgium,opp_Brazil,opp_Colombia,opp_Costa Rica,opp_Croatia,opp_Denmark,opp_Egypt,opp_England,opp_France,opp_Germany,opp_Iceland,opp_Iran,opp_Japan,opp_Korea Republic,opp_Mexico,opp_Morocco,opp_Nigeria,opp_Panama,opp_Peru,opp_Poland,opp_Portugal,opp_Russia,opp_Saudi Arabia,opp_Senegal,opp_Serbia,opp_Spain,opp_Sweden,opp_Switzerland,opp_Tunisia,opp_Uruguay
0,Russia,opp_Saudi Arabia,5,40,13,7,3,3,6,3,11,0,78,306,118,22,0,0,0,1,12.000000,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,Saudi Arabia,opp_Russia,0,60,6,0,3,3,2,1,25,2,86,511,105,10,0,0,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,Egypt,opp_Uruguay,0,43,8,3,3,2,0,1,7,3,78,395,112,12,2,0,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,Uruguay,opp_Egypt,1,57,14,4,6,4,5,1,13,3,86,589,111,6,0,0,0,1,89.000000,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,Morocco,opp_Iran,0,64,13,3,6,4,5,0,14,2,86,433,101,22,1,0,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,England,opp_Croatia,1,46,11,1,6,4,4,3,24,5,79,479,148,14,1,0,0,0,5.000000,3,0,0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [52]:
df = df_concat.drop(['Team', 'Opponent'], axis=1)

In [53]:
df

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,Man of the Match,1st Goal,Round,PSO,Goals in PSO,Argentina,Australia,Belgium,Brazil,Colombia,Costa Rica,Croatia,Denmark,Egypt,England,France,Germany,Iceland,Iran,Japan,Korea Republic,Mexico,Morocco,Nigeria,Panama,Peru,Poland,Portugal,Russia,Saudi Arabia,Senegal,Serbia,Spain,Sweden,Switzerland,Tunisia,Uruguay,opp_Argentina,opp_Australia,opp_Belgium,opp_Brazil,opp_Colombia,opp_Costa Rica,opp_Croatia,opp_Denmark,opp_Egypt,opp_England,opp_France,opp_Germany,opp_Iceland,opp_Iran,opp_Japan,opp_Korea Republic,opp_Mexico,opp_Morocco,opp_Nigeria,opp_Panama,opp_Peru,opp_Poland,opp_Portugal,opp_Russia,opp_Saudi Arabia,opp_Senegal,opp_Serbia,opp_Spain,opp_Sweden,opp_Switzerland,opp_Tunisia,opp_Uruguay
0,5,40,13,7,3,3,6,3,11,0,78,306,118,22,0,0,0,1,12.000000,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,0,60,6,0,3,3,2,1,25,2,86,511,105,10,0,0,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,0,43,8,3,3,2,0,1,7,3,78,395,112,12,2,0,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,1,57,14,4,6,4,5,1,13,3,86,589,111,6,0,0,0,1,89.000000,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0,64,13,3,6,4,5,0,14,2,86,433,101,22,1,0,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1,46,11,1,6,4,4,3,24,5,79,479,148,14,1,0,0,0,5.000000,3,0,0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
124,2,43,12,4,3,5,4,1,5,5,88,510,108,11,1,0,0,1,4.000000,4,0,0,False,False,True,False,False,

In [55]:
np.sum(df.dtypes == 'object')

np.int64(0)

### Scaling

In [56]:
y = df['Man of the Match'].copy()
X = df.drop('Man of the Match', axis=1)

In [57]:
y

0      1
1      0
2      0
3      1
4      0
      ..
123    0
124    1
125    0
126    1
127    0
Name: Man of the Match, Length: 128, dtype: int64

In [58]:
X

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,1st Goal,Round,PSO,Goals in PSO,Argentina,Australia,Belgium,Brazil,Colombia,Costa Rica,Croatia,Denmark,Egypt,England,France,Germany,Iceland,Iran,Japan,Korea Republic,Mexico,Morocco,Nigeria,Panama,Peru,Poland,Portugal,Russia,Saudi Arabia,Senegal,Serbia,Spain,Sweden,Switzerland,Tunisia,Uruguay,opp_Argentina,opp_Australia,opp_Belgium,opp_Brazil,opp_Colombia,opp_Costa Rica,opp_Croatia,opp_Denmark,opp_Egypt,opp_England,opp_France,opp_Germany,opp_Iceland,opp_Iran,opp_Japan,opp_Korea Republic,opp_Mexico,opp_Morocco,opp_Nigeria,opp_Panama,opp_Peru,opp_Poland,opp_Portugal,opp_Russia,opp_Saudi Arabia,opp_Senegal,opp_Serbia,opp_Spain,opp_Sweden,opp_Switzerland,opp_Tunisia,opp_Uruguay
0,5,40,13,7,3,3,6,3,11,0,78,306,118,22,0,0,0,12.000000,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False
1,0,60,6,0,3,3,2,1,25,2,86,511,105,10,0,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,0,43,8,3,3,2,0,1,7,3,78,395,112,12,2,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
3,1,57,14,4,6,4,5,1,13,3,86,589,111,6,0,0,0,89.000000,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,0,64,13,3,6,4,5,0,14,2,86,433,101,22,1,0,0,39.457447,0,0,0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1,46,11,1,6,4,4,3,24,5,79,479,148,14,1,0,0,5.000000,3,0,0,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
124,2,43,12,4,3,5,4,1,5,5,88,510,108,11,1,0,0,4.000000,4,0,0,False,False,True,False,False,False,False,False,False,False,False

In [59]:
scaler = RobustScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,1st Goal,Round,PSO,Goals in PSO,Argentina,Australia,Belgium,Brazil,Colombia,Costa Rica,Croatia,Denmark,Egypt,England,France,Germany,Iceland,Iran,Japan,Korea Republic,Mexico,Morocco,Nigeria,Panama,Peru,Poland,Portugal,Russia,Saudi Arabia,Senegal,Serbia,Spain,Sweden,Switzerland,Tunisia,Uruguay,opp_Argentina,opp_Australia,opp_Belgium,opp_Brazil,opp_Colombia,opp_Costa Rica,opp_Croatia,opp_Denmark,opp_Egypt,opp_England,opp_France,opp_Germany,opp_Iceland,opp_Iran,opp_Japan,opp_Korea Republic,opp_Mexico,opp_Morocco,opp_Nigeria,opp_Panama,opp_Peru,opp_Poland,opp_Portugal,opp_Russia,opp_Saudi Arabia,opp_Senegal,opp_Serbia,opp_Spain,opp_Sweden,opp_Switzerland,opp_Tunisia,opp_Uruguay
0,2.0,-0.6250,0.166667,1.166667,-0.666667,0.000000,0.333333,1.0,-0.571429,-0.666667,-0.625,-0.763770,1.6875,1.500000,-2.0,0.0,0.0,-1.168402,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.5,0.6250,-1.000000,-1.166667,-0.666667,0.000000,-1.000000,0.0,1.428571,0.000000,0.375,0.239902,0.0625,-0.500000,-2.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.5,-0.4375,-0.666667,-0.166667,-0.666667,-0.444444,-1.666667,0.0,-1.142857,0.333333,-0.625,-0.328029,0.9375,-0.166667,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.4375,0.333333,0.166667,0.333333,0.444444,0.000000,0.0,-0.285714,0.333333,0.375,0.621787,0.8125,-1.166667,-2.0,0.0,0.0,2.108194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.5,0.8750,0.166667,-0.166667,0.333333,0.444444,0.000000,-0.5,-0.142857,0.000000,0.375,-0.141983,-0.4375,1.500000,-1.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,0.0,-0.2500,-0.166667,-0.833333,0.333333,0.444444,-0.333333,1.0,1.285714,1.000000,-0.500,0.083231,5.4375,0.166667,-1.0,0.0,0.0,-1.466274,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
124,0.5,-0.4375,0.000000,0.166667,-0.666667,0.888889,-0.333333,0.0,-1.428571,1.000000,0.625,0.235006,0.4375,-0.333333,-1.0,0.0,0.0,-1.508828,16.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,

In [60]:
X.describe()

,Goal Scored,Ball Possession %,Attempts,On-Target,Off-Target,Blocked,Corners,Offsides,Free Kicks,Saves,Pass Accuracy %,Passes,Distance Covered (Kms),Fouls Committed,Yellow Card,Yellow & Red,Red,1st Goal,Round,PSO,Goals in PSO,Argentina,Australia,Belgium,Brazil,Colombia,Costa Rica,Croatia,Denmark,Egypt,England,France,Germany,Iceland,Iran,Japan,Korea Republic,Mexico,Morocco,Nigeria,Panama,Peru,Poland,Portugal,Russia,Saudi Arabia,Senegal,Serbia,Spain,Sweden,Switzerland,Tunisia,Uruguay,opp_Argentina,opp_Australia,opp_Belgium,opp_Brazil,opp_Colombia,opp_Costa Rica,opp_Croatia,opp_Denmark,opp_Egypt,opp_England,opp_France,opp_Germany,opp_Iceland,opp_Iran,opp_Japan,opp_Korea Republic,opp_Mexico,opp_Morocco,opp_Nigeria,opp_Panama,opp_Peru,opp_Poland,opp_Portugal,opp_Russia,opp_Saudi Arabia,opp_Senegal,opp_Serbia,opp_Spain,opp_Sweden,opp_Switzerland,opp_Tunisia,opp_Uruguay
count,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,1.280000e+02,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000,128.000000
mean,0.160156,-0.000488,0.098958,0.138021,0.091146,0.159722,-0.093750,0.171875,-0.015625,0.242188,-0.055664,0.003175,0.270508,0.091146,-0.304688,0.015625,0.015625,-1.040834e-17,1.937500,0.062500,0.203125,0.031250,0.023438,0.054688,0.039062,0.031250,0.023438,0.054688,0.031250,0.023438,0.054688,0.054688,0.023438,0.023438,0.023438,0.031250,0.023438,0.031250,0.023438,0.023438,0.023438,0.023438,0.023438,0.031250,0.039062,0.023438,0.023438,0.023438,0.031250,0.039062,0.031250,0.023438,0.039062,0.031250,0.023438,0.054688,0.039062,0.031250,0.023438,0.054688,0.031250,0.023438,0.054688,0.054688,0.023438,0.023438,0.023438,0.031250,0.023438,0.031250,0.023438,0.023438,0.023438,0.023438,0.023438,0.031250,0.039062,0.023438,0.023438,0.023438,0.031250,0.039062,0.031250,0.023438,0.039062
std,0.578260,0.652755,0.874304,0.744801,0.803225,1.068087,0.815357,0.596702,0.674895,0.683149,0.741721,0.740202,1.468692,0.769855,1.325454,0.124507,0.124507,8.920227e-01,4.138831,0.243013,0.807049,0.174676,0.151883,0.228263,0.194505,0.174676,0.151883,0.228263,0.174676,0.151883,0.228263,0.228263,0.151883,0.151883,0.151883,0.174676,0.151883,0.174676,0.151883,0.151883,0.151883,0.151883,0.151883,0.174676,0.194505,0.151883,0.151883,0.151883,0.174676,0.194505,0.174676,0.151883,0.194505,0.174676,0.151883,0.228263,0.194505,0.174676,0.151883,0.228263,0.174676,0.151883,0.228263,0.228263,0.151883,0.151883,0.151883,0.174676,0.151883,0.174676,0.151883,0.151883,0.151883,0.151883,0.151883,0.174676,0.194505,0.151883,0.151883,0.151883,0.174676,0.194505,0.174676,0.151883,0.194505
min,-0.500000,-1.562500,-1.500000,-1.166667,-1.333333,-1.333333,-1.666667,-0.500000,-1.428571,-0.666667,-2.000000,-1.336597,-3.062500,-1.333333,-2.000000,0.000000,0.000000,-1.636487e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [61]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

### Training

In [62]:
sk_model = MLPClassifier(hidden_layer_sizes=(32, 32))
sk_model.fit(X_train, y_train)

/home/aditya/miniconda3/envs/tfenv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:781: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


,hidden_layer_sizes,"(32, ...)"
,activation,'relu'
,solver,'adam'
,alpha,0.0001
,batch_size,'auto'
,learning_rate,'constant'
,learning_rate_init,0.001
,power_t,0.5
,max_iter,200
,shuffle,True
,random_state,None


In [69]:
inputs = tf.keras.Input(shape=(85, ))
x = tf.keras.layers.Dense(32, activation=tf.nn.relu)(inputs)
x = tf.keras.layers.Dense(32, activation=tf.nn.relu)(x)
outputs = tf.keras.layers.Dense(2, activation=tf.nn.softmax)(x)

tf_model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [70]:
tf_model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [71]:
tf_model.fit(
    X_train,
    y_train,
    validation_split = 0.2,
    batch_size = 16,
    epochs = 200
)

Epoch 1/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.4366 - loss: 0.7254 - val_accuracy: 0.6111 - val_loss: 0.6741
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.5211 - loss: 0.6888 - val_accuracy: 0.5556 - val_loss: 0.6758
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.6479 - loss: 0.6641 - val_accuracy: 0.6111 - val_loss: 0.6650
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7042 - loss: 0.6378 - val_accuracy: 0.5556 - val_loss: 0.6649
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7746 - loss: 0.6186 - val_accuracy: 0.5000 - val_loss: 0.6629
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.8028 - loss: 0.6001 - val_accuracy: 0.5000 - val_loss: 0.6609
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.8310 - loss: 0.5854 - val_accuracy: 0.5000 - val_loss: 0.6633
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8310 - loss: 0.5662 - val_accuracy: 0.5000 - val_loss:

### Results

In [75]:
sk_score = sk_model.score(X_test, y_test)
tf_score = tf_model.evaluate(X_test, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5385 - loss: 1.7854


In [77]:
print(f"   Sklearn Model: {sk_score:.3f}")
print(f"Tensorflow Model: {tf_score[1]:.3f}")

   Sklearn Model: 0.513
Tensorflow Model: 0.538
